# Converting Text to N dimension Vector By using Bag Of Words Approach


In [20]:
# Load text First
#import all neccesary lib
%matplotlib inline 


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer


#using the SQLite Table to read data
con = sqlite3.connect(r'E:\Applied AI\AppliedAICoding\Dataset\amazon-fine-food-reviews\database.sqlite')

#filtering only positive and negative reviews i.e.
#nottaking into consideration those reiviews with score=3
filtered_data =pd.read_sql_query("""
Select * from Reviews Where Score !=3""",con)

print(filtered_data.shape)


(525814, 10)


In [21]:
# Give review with Score>3 a positive rating , and reviews with a score<3 a negative rating.

def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

# changing reviews with score less than 3 to be positive and vice - versa
'''
Actual code 
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
'''

filtered_data['Score'] = filtered_data['Score'].map(partition)




In [22]:
print(filtered_data.shape)# no rows and column
filtered_data.head()


(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [26]:
display=pd.read_sql_query("""
Select * from Reviews where score !=3 and UserId="AR5J8UI46CURR" Order by productID""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [30]:
#Sorting data according to productId in ascending order

#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
sorted_data= filtered_data.sort_values('ProductId',axis=0,ascending=True, inplace=False, kind= 'quicksort',na_position='last')
sorted_data.shape

(525814, 10)

In [34]:
#Deduplication of entries
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"} , keep='first', inplace=False)
final.shape

(364173, 10)

In [35]:
#checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [43]:
display = pd.read_sql_query("""
Select * from Reviews
where Score !=3 and id=44737 or id=64422 order by productID""",con)


#or
'''Select * from Reviews where HelpfulnessNumerator>HelpfulnessDenominator'''
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [ ]:
final=f
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]